In [ ]:
# @title AnxietyLightning Launcher (v26.1 - Function Call Fix)# @markdown This cell contains the fully patched launcher. It will download the corrected backend scripts and orchestrate the setup and launch process.# @markdown **Changes:** Corrected TypeError in `update_current_webui` function call.NGROK_TOKEN = "" #@param {type:"string"}
import osimport sysfrom pathlib import Pathimport subprocessimport shleximport timeimport htmlimport reimport jsonimport runpy
# --- 0. Pre-flight Checks & Dependencies ---try:    import gradio as grexcept ImportError:    print("⚠️ Gradio not found. Installing...")    subprocess.run([sys.executable, "-m", "pip", "install", "gradio"], check=True)    import gradio as gr
# --- 1. Define Core Paths ---HOME_DIR = Path('/content')ANXETY_ROOT = HOME_DIR / 'ANXETY'SCRIPTS_DIR = ANXETY_ROOT / 'scripts'MODULES_DIR = ANXETY_ROOT / 'modules'LOG_DIR = ANXETY_ROOT / "logs"
# Set environment variables for backend scriptsos.environ['HOME_DIR'] = str(HOME_DIR)os.environ['ANXETY_ROOT'] = str(ANXETY_ROOT)
# --- 2. Download Project Files & Create ANXETY_ROOT ---print("--- 🚀 Stage 1: Synchronizing Project & Creating Root Directory ---")SETUP_PY_PATH = SCRIPTS_DIR / 'setup.py'SETUP_PY_URL = "https://raw.githubusercontent.com/drf0rk/AnxietyLightning/main/scripts/setup.py"try:    ANXETY_ROOT.mkdir(parents=True, exist_ok=True)    SCRIPTS_DIR.mkdir(parents=True, exist_ok=True)
    # Use curl to get the latest setup script, then run it to get the rest of the repo    # This ensures we always run the latest code, including the fixes.    subprocess.run(["curl", "-sL", SETUP_PY_URL, "-o", str(SETUP_PY_PATH)], check=True, text=True)    subprocess.run([sys.executable, str(SETUP_PY_PATH)], check=True)    print("✅ Project files synchronized with the latest version from GitHub.")
    LOG_DIR.mkdir(exist_ok=True)    print(f"✅ Log directory ensured at {LOG_DIR}")
except subprocess.CalledProcessError as e:    print(f"❌ FATAL: Project setup failed during sync. Return code: {e.returncode}. Error: {e.stderr}")    sys.exit(1)except Exception as e:    print(f"❌ FATAL: Project setup failed. Error: {e}")    sys.exit(1)
# Add project to Python's path to ensure our modules are foundif str(ANXETY_ROOT) not in sys.path: sys.path.insert(0, str(ANXETY_ROOT))
try:    from modules import json_utils as js    from modules import webui_utils # Import the new module    print("✅ Core modules imported.")except ImportError as e:    print(f"❌ FATAL: Failed to import project modules after sync. Check if files exist in {MODULES_DIR}. Error: {e}")    sys.exit(1)
# --- 3. Data Loading (Directly in Cell) ---print("\n--- 🚀 Stage 2: Loading Asset Data ---")try:    sd15_data = runpy.run_path(str(ANXETY_ROOT / 'scripts/_models-data.py'))    sdxl_data = runpy.run_path(str(ANXETY_ROOT / 'scripts/_xl-models-data.py'))    loras_data_full = runpy.run_path(str(ANXETY_ROOT / 'scripts/_loras-data.py'))    sd15_model_choices = list(sd15_data.get('sd15_model_data', {}).keys())    sd15_vae_choices = list(sd15_data.get('sd15_vae_data', {}).keys())    sd15_controlnet_choices = list(sd15_data.get('controlnet_list', {}).keys())    sd15_lora_choices = list(loras_data_full.get('lora_data', {}).get('sd15_loras', {}).keys())    sdxl_model_choices = list(sdxl_data.get('sdxl_models_data', {}).keys())    sdxl_vae_choices = list(sdxl_data.get('sdxl_vae_data', {}).keys())    sdxl_controlnet_choices = list(sdxl_data.get('controlnet_list', {}).keys())    sdxl_lora_choices = list(loras_data_full.get('lora_data', {}).get('sdxl_loras', {}).keys())    print("✅ Asset data loaded.")except Exception as e:    print(f"❌ FATAL: Could not load asset data files. Error: {e}")    sys.exit(1)
# --- Default Arguments and UI Styling ---webui_selection_args = {    'A1111': "--xformers --no-half-vae --skip-torch-cuda-test --reinstall-xformers",    'Forge': "--disable-xformers --opt-sdp-attention --cuda-stream --pin-shared-memory --skip-torch-cuda-test",    'ReForge': "--disable-xformers --cuda-stream --pin-shared-memory --skip-torch-cuda-test",    'Classic': "--persistent-patches --cuda-stream --skip-torch-cuda-test --reinstall-xformers",    'ComfyUI': "--use-sage-attention",    'SD-UX': "--xformers --no-half-vae --skip-torch-cuda-test --reinstall-xformers"}MODERN_LOG_CSS="""<style>#log_output_html{background-color:#0d1117;border:1px solid #30363d;border-radius:8px;padding:12px;font-family:'Monaco','Consolas','Menlo',monospace;color:#c9d1d9;height:400px;overflow-y:auto}#log_output_html .log-line{display:block;animation:fadeIn .5s ease-in-out; white-space: pre-wrap; word-break: break-all;}#log_output_html .log-header{color:#58a6ff;font-weight:700;margin-top:15px;margin-bottom:5px;text-shadow:0 0 5px rgba(88,166,255,.3)}#log_output_html .log-success{color:#3fb950}#log_output_html .log-error{color:#f85149;font-weight:700}#log_output_html .log-warning{color:#d29922;font-weight:700}#log_output_html .log-url{color:#9e87ff;font-weight:700}#log_output_html .log-download{color:#8b949e;font-size:.85em}@keyframes fadeIn{from{opacity:0;transform:translateY(5px)}to{opacity:1;transform:translateY(0)}}</style>"""
# --- 4. Gradio UI Definition & Backend Logic ---print("\n--- 🚀 Stage 3: Defining Gradio UI ---")
def _serialize_settings_to_json(webui_choice, is_sdxl, selected_models, selected_vaes, selected_loras, selected_cnets, launch_args, ngrok_token_val, detailed_download):    final_launch_args = launch_args    default_args_for_ui = webui_selection_args.get(webui_choice, "")
    # Basic argument merging (could be more robust, but works for now)    if not final_launch_args:        final_launch_args = default_args_for_ui    else:        # Ensure critical flags are present if they are in the default but not user input        default_flags = default_args_for_ui.split()        user_flags = final_launch_args.split()        for flag in default_flags:            if flag not in user_flags:                if flag == '--disable-xformers' and '--xformers' in user_flags:                    continue # User explicitly wants xformers, ignore disable                if flag == '--xformers' and '--disable-xformers' in user_flags:                    continue # User explicitly wants to disable, ignore xformers                final_launch_args += f" {flag}"
    # Clean up potential duplicates    final_launch_args = ' '.join(list(dict.fromkeys(final_launch_args.split())))
    settings_data = {'WIDGETS':{'change_webui':webui_choice,'sdxl_toggle':is_sdxl,'model_list':selected_models or [],'vae_list':selected_vaes or [],'lora_list':selected_loras or [],'controlnet_list':selected_cnets or [],'commandline_arguments':final_launch_args,'ngrok_token':ngrok_token_val or NGROK_TOKEN,'detailed_download':detailed_download},'ENVIRONMENT':{'home_path':str(HOME_DIR)}}    settings_path = ANXETY_ROOT / 'settings.json'    js.save(str(settings_path), 'WIDGETS', settings_data['WIDGETS'])    js.save(str(settings_path), 'ENVIRONMENT', settings_data['ENVIRONMENT'])    # Use the new utility to save the current context    webui_utils.update_current_webui(webui_choice)
def format_log_line(line):    """Formats raw output lines into HTML for the Gradio log viewer."""    clean_line = line.strip()    if not clean_line: return None
    try:        # Try parsing as structured JSON log from our logging_utils        log_entry = json.loads(clean_line)        level, message = log_entry.get('level', 'info'), html.escape(log_entry.get('message', ''))                if level == 'progress': # Special handling for download progress             raw_data = html.escape(log_entry.get('data', {}).get('raw_line', ''))             percent = log_entry.get('data', {}).get('percentage', 0)             message = f"🔄 {message} ({percent}%) {raw_data}"             return f'<span class="log-line log-download progress-line">{message}</span>', True                return f'<span class="log-line log-{level}">{message}</span>', False
    except json.JSONDecodeError:        # Handle raw text output (e.g., from WebUI startup)        escaped_line = html.escape(clean_line)        if '%' in escaped_line and ('MiB/s' in escaped_line or 'ETA' in escaped_line):             return f'<span class="log-line log-download progress-line">🔄 {escaped_line}</span>', True        elif any(x in escaped_line for x in ["gradio.live", ".trycloudflare.com", "ngrok-free.app"]):            return f'<span class="log-line log-url">🔗 {escaped_line}</span>', False        else:            return f'<span class="log-line">{escaped_line}</span>', False
def save_and_launch(webui_choice, is_sdxl, selected_models, selected_vaes, selected_loras, selected_cnets, launch_args, ngrok_token_val, detailed_download):    _serialize_settings_to_json(webui_choice, is_sdxl, selected_models, selected_vaes, selected_loras, selected_cnets, launch_args, ngrok_token_val, detailed_download)    log_lines_list = ['<span class="log-line log-success">✅ Settings saved. Starting backend setup...</span>']    yield "".join(log_lines_list)    time.sleep(0.5)
    scripts_to_run = [ANXETY_ROOT/'scripts'/'en'/'downloading-en.py', ANXETY_ROOT/'scripts'/'launch.py']        for script_path in scripts_to_run:        log_lines_list.append(f'<span class="log-line log-header">--- 🚀 Running {script_path.name} ---</span>')        yield "".join(log_lines_list)                try:            # Run scripts using the base Colab python, as they manage their own VENV usage            process = subprocess.Popen([sys.executable, str(script_path)], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, encoding='utf-8', errors='replace', bufsize=1)                        for line in iter(process.stdout.readline, ''):                html_line, is_progress = format_log_line(line)                if html_line:                    # Check if the last line was also a progress line                    last_was_progress = 'progress-line' in (log_lines_list[-1] if log_lines_list else "")
                    if is_progress and last_was_progress:                        # Replace the last progress line with the new one                        log_lines_list[-1] = html_line                    elif '✅✅✅' in html_line and last_was_progress:                        # If it's a final success message after progress, remove last progress line first                        log_lines_list.pop()                        log_lines_list.append(html_line)                    else:                        log_lines_list.append(html_line)                                        # Limit history to prevent browser lag                    if len(log_lines_list) > 500: log_lines_list = log_lines_list[-400:]
                    yield "".join(log_lines_list)
            process.wait()            if process.returncode != 0:                log_lines_list.append(f'<span class="log-line log-error">--- ❌ {script_path.name} failed with exit code {process.returncode}. Halting. ---</span>')                yield "".join(log_lines_list); break                except Exception as e:            log_lines_list.append(f'<span class="log-line log-error">--- ❌ CRITICAL FAILURE ({script_path.name}): {html.escape(str(e))} ---</span>')            yield "".join(log_lines_list); break
    # This message will only show if the loop completes without breaking    if process.returncode == 0:       log_lines_list.append('<span class="log-line log-success">--- ✅ Process Complete or Terminated Successfully ---</span>')       yield "".join(log_lines_list)
# --- Gradio UI Layout ---with gr.Blocks(theme=gr.themes.Soft(), css=MODERN_LOG_CSS) as demo:    gr.Markdown("## ⚡ AnxietyLightning Launcher (Resilient Pathing Edition)")    with gr.Tabs():        with gr.TabItem("1. Setup & Asset Selection"):            with gr.Row():                webui_dropdown = gr.Dropdown(choices=['ReForge', 'Forge', 'A1111', 'ComfyUI', 'Classic', 'SD-UX'], value='ReForge', label="Select WebUI")                sdxl_toggle = gr.Checkbox(label="Use SDXL Models", value=False)            with gr.Accordion("Asset Selection", open=True):                with gr.Row():                    model_checkboxes = gr.CheckboxGroup(choices=sd15_model_choices, label="Checkpoints", interactive=True)                    vae_checkboxes = gr.CheckboxGroup(choices=sd15_vae_choices, label="VAEs", interactive=True)                with gr.Row():                    lora_checkboxes = gr.CheckboxGroup(choices=sd15_lora_choices, label="LoRAs", interactive=True)                    controlnet_checkboxes = gr.CheckboxGroup(choices=sd15_controlnet_choices, label="ControlNets", interactive=True)            with gr.Accordion("Advanced Options", open=False):                args_textbox = gr.Textbox(label="Commandline Arguments", value=webui_selection_args['ReForge'], lines=2, interactive=True)                with gr.Row():                    ngrok_textbox_ui = gr.Textbox(label="NGROK Token", type="password", scale=3, value=NGROK_TOKEN)                    detailed_dl_checkbox = gr.Checkbox(label="Detailed Logs", value=False, scale=1)                with gr.TabItem("2. Launch & Live Log"):            launch_button = gr.Button("Install, Download & Launch", variant="primary")            output_log = gr.HTML(label="Live Log", elem_id="log_output_html")
    # --- Gradio Event Handlers ---    def update_asset_choices_local(is_sdxl):        models, vaes, loras, cnets = (sdxl_model_choices, sdxl_vae_choices, sdxl_lora_choices, sdxl_controlnet_choices) if is_sdxl else (sd15_model_choices, sd15_vae_choices, sd15_lora_choices, sd15_controlnet_choices)        return [gr.update(choices=models, value=[]), gr.update(choices=vaes, value=[]), gr.update(choices=loras, value=[]), gr.update(choices=cnets, value=[])]
    def update_args_local(webui_choice_val):        return gr.update(value=webui_selection_args.get(webui_choice_val, ""))
    sdxl_toggle.change(fn=update_asset_choices_local, inputs=sdxl_toggle, outputs=[model_checkboxes, vae_checkboxes, lora_checkboxes, controlnet_checkboxes])    webui_dropdown.change(fn=update_args_local, inputs=webui_dropdown, outputs=args_textbox)        # The main launch trigger    launch_button.click(        fn=save_and_launch,        inputs=[webui_dropdown, sdxl_toggle, model_checkboxes, vae_checkboxes, lora_checkboxes, controlnet_checkboxes, args_textbox, ngrok_textbox_ui, detailed_dl_checkbox],        outputs=[output_log]    )
# --- 5. Launch Gradio ---print("\n--- 🚀 Stage 4: Launching Gradio UI ---")# Use queue for better handling of streaming outputdemo.queue().launch(share=True, inline=False, debug=True)

--- 🚀 Stage 1: Synchronizing Project & Creating Root Directory ---
✅ Project files synchronized with the latest version from GitHub.
✅ Log directory ensured at /content/ANXETY/logs
✅ Core modules imported.

--- 🚀 Stage 2: Loading Asset Data ---
✅ Asset data loaded.

--- 🚀 Stage 3: Defining Gradio UI ---

--- 🚀 Stage 4: Launching Gradio UI ---
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://<your-url>.gradio.live
